In [1]:
#load all necessary libraries
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
#read the dataset
airfields_df = pd.read_csv('Resources/Airfields.csv') 
airfields_df.head()

,Name,Country,Archipelago,Latitude,Longitude,Type,Military,Runway_1,Runway_2,Surface,Class
0,Rota Intl Airport,U.S.,Mariana Islands,14.172008,145.243761,Air taxi,No,7000,0,Asphalt,Class_1
1,Angaur Airfield,Palau,Caroline Islands,6.900841,134.139090,Air taxi,No,7000,0,Gravel,Class_1
2,Sanga-Sanga Airport,Philippines,Sulu Archipelago,5.045266,119.742981,Air taxi,No,6102,0,Concrete,Class_1
3,Jolo Airport,Philippines,Sulu Archipelago,6.052973,121.005534,Air taxi,No,6053,0,Asphalt,Class_1
4,Peleliu Airfield,Palau,Caroline Islands,6.998333,134.232778,Air taxi,No,6000,0,Gravel,Class_1


In [3]:
# Bin runway surface values with Replace function 

airfields_df = airfields_df.replace(['Asphalt','Concrete','Asphalt/Concrete'],'Paved')
airfields_df = airfields_df.replace(['Coral','Macadam'],'Hard')
airfields_df = airfields_df.replace(['Turf/gravel'],'Gravel')
airfields_df = airfields_df.replace(['Turf'],'Grass')
airfields_df.head()

,Name,Country,Archipelago,Latitude,Longitude,Type,Military,Runway_1,Runway_2,Surface,Class
0,Rota Intl Airport,U.S.,Mariana Islands,14.172008,145.243761,Air taxi,No,7000,0,Paved,Class_1
1,Angaur Airfield,Palau,Caroline Islands,6.900841,134.139090,Air taxi,No,7000,0,Gravel,Class_1
2,Sanga-Sanga Airport,Philippines,Sulu Archipelago,5.045266,119.742981,Air taxi,No,6102,0,Paved,Class_1
3,Jolo Airport,Philippines,Sulu Archipelago,6.052973,121.005534,Air taxi,No,6053,0,Paved,Class_1
4,Peleliu Airfield,Palau,Caroline Islands,6.998333,134.232778,Air taxi,No,6000,0,Gravel,Class_1


In [4]:
airfields_size=airfields_df['Class'].value_counts()
airfields_type=airfields_df['Type'].value_counts()
print(airfields_size)
print(airfields_type)

Class_0    95
Class_1    44
Class_2    18
Class_3     7
Name: Class, dtype: int64
Unimproved    66
Air taxi      46
Commercial    30
General       22
Name: Type, dtype: int64


In [5]:
airfields_params_df = airfields_df.drop(['Name','Country','Archipelago','Latitude','Longitude','Military'], axis=1)
airfields_params_df.head()

,Type,Runway_1,Runway_2,Surface,Class
0,Air taxi,7000,0,Paved,Class_1
1,Air taxi,7000,0,Gravel,Class_1
2,Air taxi,6102,0,Paved,Class_1
3,Air taxi,6053,0,Paved,Class_1
4,Air taxi,6000,0,Gravel,Class_1


In [6]:
airfields_cat_params_df = airfields_params_df.drop(['Runway_1','Runway_2','Class'], axis=1)
airfields_cat_params_df.head()

,Type,Surface
0,Air taxi,Paved
1,Air taxi,Gravel
2,Air taxi,Paved
3,Air taxi,Paved
4,Air taxi,Gravel


In [7]:
# Generate our categorical variable lists
airfield_cats = airfields_cat_params_df.dtypes[airfields_cat_params_df.dtypes == "object"].index.tolist()
airfield_cats

['Type', 'Surface']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(airfields_cat_params_df[airfield_cats]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(airfield_cats)
encode_df.head()

,Type_Air taxi,Type_Commercial,Type_General,Type_Unimproved,Surface_Grass,Surface_Gravel,Surface_Hard,Surface_Paved
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(airfields_cat_params_df[airfield_cats]))

In [10]:
# Rename encoded columns
encode_df.columns = enc.get_feature_names_out(airfield_cats)
encode_df.head()

,Type_Air taxi,Type_Commercial,Type_General,Type_Unimproved,Surface_Grass,Surface_Gravel,Surface_Hard,Surface_Paved
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
# Separate the Runway data
airfields_runways_df = airfields_params_df.drop(['Type','Surface','Class'], axis=1)
airfields_runways_df.head()

,Runway_1,Runway_2
0,7000,0
1,7000,0
2,6102,0
3,6053,0
4,6000,0


In [12]:
# Normalize the Runway data
airfields_runways_norm_df = (airfields_runways_df-airfields_runways_df.min())/(airfields_runways_df.max()-airfields_runways_df.min())
airfields_runways_norm_df.head()

,Runway_1,Runway_2
0,0.536364,0.0
1,0.536364,0.0
2,0.454727,0.0
3,0.450273,0.0
4,0.445455,0.0


In [13]:
airfields_class_df = airfields_params_df.drop(['Runway_1','Runway_2','Surface','Type'], axis=1)
airfields_class_df.head()

,Class
0,Class_1
1,Class_1
2,Class_1
3,Class_1
4,Class_1


In [14]:
# Merge the Class and normalized Runway data
airfields_params2_df = airfields_class_df.join(airfields_runways_norm_df)
airfields_params2_df.head()

,Class,Runway_1,Runway_2
0,Class_1,0.536364,0.0
1,Class_1,0.536364,0.0
2,Class_1,0.454727,0.0
3,Class_1,0.450273,0.0
4,Class_1,0.445455,0.0


In [15]:
# Merge the Class, Runway, and encoded data
airfields_newparams_df = airfields_params2_df.join(encode_df)
airfields_newparams_df.head()

,Class,Runway_1,Runway_2,Type_Air taxi,Type_Commercial,Type_General,Type_Unimproved,Surface_Grass,Surface_Gravel,Surface_Hard,Surface_Paved
0,Class_1,0.536364,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Class_1,0.536364,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Class_1,0.454727,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Class_1,0.450273,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Class_1,0.445455,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
#Create training and test datasets

X = airfields_newparams_df.drop(['Class'], axis=1) 
y = airfields_newparams_df['Class']

print(list(X.columns.values)) 

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

['Runway_1', 'Runway_2', 'Type_Air taxi', 'Type_Commercial', 'Type_General ', 'Type_Unimproved', 'Surface_Grass', 'Surface_Gravel', 'Surface_Hard', 'Surface_Paved']
(131, 10)
(33, 10)
(131,)
(33,)


In [18]:
model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

#print the tunable parameters (They were not tuned in this example, everything kept as default)
params = model1.get_params()
print(params)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'none', 'random_state': 0, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [19]:
#Print model parameters
print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

Intercept: 
 [ 122.70554694  117.41321461 -190.28172641  -49.83703548]
Coefficients: 
 [[-3.79552104e+02 -3.41924607e+02  1.92563448e+01  1.42911376e+01
   7.39766415e+01  1.51814231e+01  7.46106066e+00  3.89667183e+01
   2.51546452e+00  7.37623034e+01]
 [-3.49570695e+02 -3.48514779e+02  1.61112846e+01  1.96669163e+01
   7.26909765e+01  8.94403716e+00  7.54260006e+00  3.66136627e+01
   3.01340685e-01  7.29556112e+01]
 [ 6.15501535e+02  2.78415338e+02 -3.26615876e+01 -8.00670036e+01
  -3.20383452e+01 -4.55147900e+01 -3.87037317e-01 -7.46303393e+01
  -1.72145647e+01 -9.80497851e+01]
 [ 1.13621264e+02  4.12024049e+02 -2.70604205e+00  4.61089501e+01
  -1.14629273e+02  2.13893295e+01 -1.46166235e+01 -9.50041792e-01
   1.43977595e+01 -4.86681297e+01]]


In [20]:
#Calculate odds ratio estimates
import numpy as np
np.exp(model1.coef_)

array([[1.45417182e-165, 3.19175795e-149, 2.30634499e+008,
        1.60902274e+006, 1.34167464e+032, 3.91929787e+006,
        1.73899157e+003, 8.37588597e+016, 1.23723547e+001,
        1.08283265e+032],
       [1.52537338e-152, 4.38484767e-152, 9.93212068e+006,
        3.47722855e+008, 3.70928322e+031, 7.66206771e+003,
        1.88672926e+003, 7.96364098e+015, 1.35166976e+000,
        4.83302847e+031],
       [2.03666758e+267, 8.20814096e+120, 6.53510760e-015,
        1.68788220e-035, 1.21877479e-014, 1.71071584e-020,
        6.79065754e-001, 3.87663975e-033, 3.34048526e-008,
        2.61528715e-043],
       [2.21354323e+049, 8.70504083e+178, 6.68006777e-002,
        1.05891861e+020, 1.64869105e-050, 1.94655987e+009,
        4.48829181e-007, 3.86724861e-001, 1.79005959e+006,
        7.30633922e-022]])

In [22]:
#Use statsmodels to assess variables

logit_model=sm.MNLogit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

Optimization terminated successfully.
         Current function value: nan
         Iterations 14
                          MNLogit Regression Results                          
Dep. Variable:                  Class   No. Observations:                  131
Model:                        MNLogit   Df Residuals:                      104
Method:                           MLE   Df Model:                           24
Date:                Wed, 14 Sep 2022   Pseudo R-squ.:                     nan
Time:                        20:34:27   Log-Likelihood:                    nan
converged:                       True   LL-Null:                       -135.93
Covariance Type:            nonrobust   LLR p-value:                       nan
  Class=Class_1       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const                  nan        nan        nan        nan         nan         nan
Runway_1          

C:\Users\drchr\anaconda3\envs\mlenv\lib\site-packages\statsmodels\discrete\discrete_model.py:2299: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\drchr\anaconda3\envs\mlenv\lib\site-packages\statsmodels\discrete\discrete_model.py:2300: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]


In [23]:
#Create a confusion matrix
#y_test as first argument and the preds as second argument 
confusion_matrix(y_test, preds)

array([[19,  1,  0,  0],
       [ 0,  6,  2,  0],
       [ 0,  0,  2,  0],
       [ 0,  0,  2,  1]], dtype=int64)

In [24]:
#transform confusion matrix into array
#the matrix is stored in a vaiable called confmtrx
confmtrx = np.array(confusion_matrix(y_test, preds))
#Create DataFrame from confmtrx array 
#rows for test: Male, Female, Infant designation as index 
#columns for preds: male, predicted_female, predicted_infant as column

pd.DataFrame(confmtrx, index=['Class_0','Class_1','Class_2','Class_3'],
columns=['predicted_Class_0','predicted_Class_1','predicted_Class_2','predicted_Class_3'])

,predicted_Class_0,predicted_Class_1,predicted_Class_2,predicted_Class_3
Class_0,19,1,0,0
Class_1,0,6,2,0
Class_2,0,0,2,0
Class_3,0,0,2,1


In [25]:
#Accuracy statistics

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

Accuracy Score: 0.8484848484848485


In [26]:
#Create classification report
class_report=classification_report(y_test, preds)
print(class_report)

              precision    recall  f1-score   support

     Class_0       1.00      0.95      0.97        20
     Class_1       0.86      0.75      0.80         8
     Class_2       0.33      1.00      0.50         2
     Class_3       1.00      0.33      0.50         3

    accuracy                           0.85        33
   macro avg       0.80      0.76      0.69        33
weighted avg       0.92      0.85      0.86        33

